In [1]:
import os
import yaml
import pandas as pd
from geoalchemy2 import Geometry
from sqlalchemy import create_engine, inspect
import geopandas as gpd

# yaml파일 로드
납품 폴더에 대한 필드 정의 및 시스템 폴더에 대한 법정동코드 정의

In [2]:
with open('fields.yaml', encoding='utf-8') as f:
    fc = yaml.safe_load(f)
INST_FIELDS     = fc['기관용']
SYS_FIELDS      = fc['시스템용']
SYS_HIST_FIELDS = fc['시스템용_hist']
NJ_FIELDS       = fc['농정원용']

# 0.5) code_mapping.yaml 로드
with open('code_mapping.yaml', encoding='utf-8') as f:
    code_map = yaml.safe_load(f)

def get_code(tbl):
    _, sido, sigungu = tbl.split('_', 2)
    return code_map[sido][sigungu]

# DB 접속
FDBMS의 데이터베이스 Farmap_Data 접속

In [3]:
db_info = {
    'host':     '192.168.49.171',
    'port':     62484,
    'database': 'farmmap_demo',
    'user':     'Opt-Ai',
    'password': 'opt-ai'
}
engine = create_engine(
    f"postgresql+psycopg2://{db_info['user']}:{db_info['password']}"
    f"@{db_info['host']}:{db_info['port']}/{db_info['database']}"
)

# DB 스키마 및 validation suffix
1. produciton_db 및 validation_db 접속
2. val_suffixes = validation_db의 변경, 삭제만

In [4]:
schemas      = {'prod':'production_db','val':'validation_db'}
val_suffixes = ['_변경','_삭제']    # 신규는 기관/농정원용에만 필요하다면 따로 처리

# 메타데이터 캐싱
1. inspcet(engine) = 데이터베이스 연결에 대한 inspect 객체 생성
2. get_table_names = 스키마에 속한 테이블 이름 파싱

In [5]:
inspector   = inspect(engine)
prod_tables = inspector.get_table_names(schema=schemas['prod'])
val_tables  = set(inspector.get_table_names(schema=schemas['val']))

columns_cache = {
    'prod': {t: inspector.get_columns(schema=schemas['prod'], table_name=t) for t in prod_tables},
    'val' : {t: inspector.get_columns(schema=schemas['val'],  table_name=t) for t in val_tables}
}


# Shapefile 내보내기(export_one)

1. `columns_cache`에서 대상 테이블의 컬럼 메타데이터 조회  
2. (‘geom’, ‘geometry’, ‘the_geom’) 중 geometry 컬럼 자동 탐색  
3. 지정된 `fields` + geometry 컬럼으로 PostGIS에서 GeoDataFrame 로드  
4. 중복 컬럼 제거  
5. (`add_file_nm=True`) 파일명 컬럼 추가·재정렬  
6. ESRI Shapefile로 저장 후 geometry 컬럼명 반환 

In [6]:
def export_one(schema, tbl, outdir, fname, fields, add_file_nm=False):
    cols = columns_cache[schema].get(tbl)
    if not cols:
        return None
    actual   = [c['name'] for c in cols]
    m        = {c.lower():c for c in actual}
    geom_col = next((m[k] for k in ('geom','geometry','the_geom') if k in m), None)
    if geom_col is None:
        return None
    sel      = [m[f.lower()] for f in fields if f.lower() in m]
    cols_sql = (sel or []) + [geom_col]
    q        = ','.join(f'"{c}"' for c in cols_sql)
    sql      = f'SELECT {q} FROM "{schemas[schema]}"."{tbl}"'
    gdf      = gpd.read_postgis(sql, engine, geom_col=geom_col)
    gdf      = gdf.loc[:, ~gdf.columns.duplicated()]  # 중복 제거

    if add_file_nm:
        zipname = os.path.splitext(fname)[0] + ".zip"
        gdf['FILE_NM'] = zipname
        gdf = gdf.loc[:, ~gdf.columns.duplicated()]
        order = [c for c in SYS_FIELDS if c in gdf.columns] + ['FILE_NM'] + [geom_col]
        gdf = gdf[order]

    out_path = os.path.join(outdir, fname)
    gdf.to_file(out_path, driver='ESRI Shapefile')
    return geom_col

# 전체 처리 흐름 (prod_tables 순회)
1. 테이블별 폴더 구조 생성  
   - `./<tbl>/기관용`  
   - `./<tbl>/시스템용/farm_map_dmnd_<code>`  
   - `./<tbl>/시스템용/farm_map_dmnd_hist_<code>`  
   - `./<tbl>/농정원용`

2. 기관용  
   - `export_one('prod', tbl, 기관용, tbl.shp, INST_FIELDS)`  
   - val 스키마의 `_신규`, `_변경`, `_삭제` 테이블이 있으면 모두 export_one 실행

3. 시스템용  
   a. **원본** (`prod`만)  
      - `export_one('prod', tbl, farm_map_dmnd_<code>, farm_map_dmnd_<code>.shp, SYS_FIELDS, add_file_nm=True)`  
   b. **히스토리** (`val`의 `_변경`+`_삭제`)  
      - 각 suffix 테이블에서 SYS_HIST_FIELDS+geometry 로 GeoDataFrame 로드  
      - concat → 누락 필드 채움 → `FILE_NM` 컬럼 추가 → 필드 순서 재정렬  
      - Shapefile로 저장

4. 농정원용  
   a. **원본** (`prod`만)  
      - `export_one('prod', tbl, 농정원용, tbl.shp, NJ_FIELDS)`  
   b. **히스토리** (`val`의 `_변경`+`_삭제`)  
      - NJ_FIELDS+geometry 로 GeoDataFrame 로드 → concat → Shapefile로 저장

In [7]:
base = os.getcwd()

for tbl in prod_tables:
    # 테이블별 최상위 폴더
    tbl_root = os.path.join(base, tbl)
    os.makedirs(tbl_root, exist_ok=True)

    # 1) 기관용
    inst_dir = os.path.join(tbl_root, "기관용")
    os.makedirs(inst_dir, exist_ok=True)
    export_one('prod', tbl,     inst_dir, f"{tbl}.shp",     INST_FIELDS)
    # 신규/변경/삭제 모두 내보이려면:
    for suf in ['_신규','_변경','_삭제']:
        vt = tbl + suf
        if vt in val_tables:
            export_one('val', vt, inst_dir, f"{vt}.shp", INST_FIELDS)

    # 2) 시스템용
    sys_root = os.path.join(tbl_root, "시스템용")
    os.makedirs(sys_root, exist_ok=True)
    code     = get_code(tbl)
    dmnd_dir = os.path.join(sys_root, f"farm_map_dmnd_{code}")
    hist_dir = os.path.join(sys_root, f"farm_map_dmnd_hist_{code}")
    os.makedirs(dmnd_dir, exist_ok=True)
    os.makedirs(hist_dir, exist_ok=True)

    # 시스템용 원본 (prod only + FILE_NM)
    export_one('prod', tbl, dmnd_dir,
               f"farm_map_dmnd_{code}.shp", SYS_FIELDS, add_file_nm=True)

    # 시스템용 히스토리 (_변경 + _삭제 병합 + FILE_NM)
    parts = []
    for suf in val_suffixes:
        vt = tbl + suf
        if vt in val_tables:
            cols    = columns_cache['val'][vt]
            actual  = [c['name'] for c in cols]
            m2      = {c.lower():c for c in actual}
            geom2   = next((m2[k] for k in ('geom','geometry','the_geom') if k in m2), None)
            sel2    = [m2[f.lower()] for f in SYS_HIST_FIELDS if f.lower() in m2]
            cols2   = (sel2 or []) + [geom2]
            q2      = ','.join(f'"{c}"' for c in cols2)
            sql2    = f'SELECT {q2} FROM "{schemas["val"]}"."{vt}"'
            parts.append(gpd.read_postgis(sql2, engine, geom_col=geom2))

    if parts:
        merged = gpd.GeoDataFrame(pd.concat(parts, ignore_index=True, sort=False),
                                  crs=parts[0].crs)
        for fld in SYS_HIST_FIELDS:
            if fld not in merged.columns:
                merged[fld] = None
        zipname = f"farm_map_dmnd_hist_{code}.zip"
        merged['FILE_NM'] = zipname
        merged = merged.loc[:, ~merged.columns.duplicated()]
        merged = merged[ SYS_HIST_FIELDS + ['FILE_NM'] + [merged.geometry.name] ]
        merged.to_file(os.path.join(hist_dir, f"farm_map_dmnd_hist_{code}.shp"),
                       driver='ESRI Shapefile')

    # 3) 농정원용
    nj_dir = os.path.join(tbl_root, "농정원용")
    os.makedirs(nj_dir, exist_ok=True)
    export_one('prod', tbl, nj_dir, f"{tbl}.shp", NJ_FIELDS)
    parts_nj = []
    for suf in ['_변경','_삭제']:
        vt = tbl + suf
        if vt in val_tables:
            cols   = columns_cache['val'][vt]
            actual = [c['name'] for c in cols]
            m3     = {c.lower():c for c in actual}
            geom3  = next((m3[k] for k in ('geom','geometry','the_geom') if k in m3), None)
            sel3   = [m3[f.lower()] for f in NJ_FIELDS if f.lower() in m3]
            cols3  = (sel3 or []) + [geom3]
            q3     = ','.join(f'"{c}"' for c in cols3)
            sql3   = f'SELECT {q3} FROM "{schemas["val"]}"."{vt}"'
            parts_nj.append(gpd.read_postgis(sql3, engine, geom_col=geom3))

    if parts_nj:
        merged_nj = gpd.GeoDataFrame(pd.concat(parts_nj, ignore_index=True, sort=False),
                                     crs=parts_nj[0].crs)
        merged_nj = merged_nj[NJ_FIELDS + [merged_nj.geometry.name]]
        merged_nj.to_file(os.path.join(nj_dir, f"{tbl}_hist.shp"),
                          driver='ESRI Shapefile')